In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

In [0]:
def mnist_dataset():
  (x, y), _ = datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

In [0]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

In [0]:
model = keras.Sequential([
    layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    layers.Dense(100, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(10)])

optimizer = optimizers.Adam()

In [0]:
@tf.function
def compute_loss(logits, labels):
  return tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits, labels=labels))

In [0]:
@tf.function
def compute_accuracy(logits, labels):
  predictions = tf.argmax(logits, axis=1)
  return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

In [0]:
@tf.function
def train_one_step(model, optimizer, x, y):

  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(logits, y)

  # compute gradient
  grads = tape.gradient(loss, model.trainable_variables)
  # update to weights
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  accuracy = compute_accuracy(logits, y)

  # loss and accuracy is scalar tensor
  return loss, accuracy

In [0]:
def train(epoch, model, optimizer):

  train_ds = mnist_dataset()
  loss = 0.0
  accuracy = 0.0
  for step, (x, y) in enumerate(train_ds):
    loss, accuracy = train_one_step(model, optimizer, x, y)

    if step % 500 == 0:
      print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

  return loss, accuracy

In [21]:
#Now that we have our training procedure set up, we can throw it in a loop and start training!
for epoch in range(10):
  loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.350637 ; accuracy 0.11
epoch 1 : loss 0.1645571 ; accuracy 0.96
epoch 2 : loss 0.09067838 ; accuracy 0.99
epoch 3 : loss 0.07104866 ; accuracy 0.98
epoch 4 : loss 0.042906303 ; accuracy 0.99
epoch 5 : loss 0.026218753 ; accuracy 1.0
epoch 6 : loss 0.04308651 ; accuracy 0.99
epoch 7 : loss 0.055087253 ; accuracy 0.99
epoch 8 : loss 0.05955946 ; accuracy 0.99
epoch 9 : loss 0.0084615685 ; accuracy 1.0
Final epoch 9 : loss 0.030388586 ; accuracy 0.99
